In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load JSON data
with open('data/watch-history.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.json_normalize(data)

# Show the first few rows
df.head()

In [ ]:
df.columns

In [ ]:
#Change the time format 
df['time'] = pd.to_datetime(df['time'], format='ISO8601', errors='coerce', utc=True)


In [ ]:
# Extract useful time features
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['weekday'] = df['time'].dt.day_name()   # Saturday, Sunday,....etc
df['hour'] = df['time'].dt.hour

In [ ]:
#visualization of watch hours during the day
watch_by_hour = df['hour'].value_counts().sort_index()

plt.figure(figsize=(10,5))
plt.bar(watch_by_hour.index, watch_by_hour.values, align='center', width=0.7)
plt.xticks(range(24))
plt.xlabel('Hour (24-hour)')
plt.ylabel('Number of Videos Watched')
plt.title('YouTube Watch Activity by Hour of Day')
plt.show()


In [ ]:
#count of watched videos on weekdays(sorted)
watch_by_weekday = df['weekday'].value_counts().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

print(watch_by_weekday)

In [ ]:
#Visualization of watch activity by weekday
df['weekday'] = df['time'].dt.day_name()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
watch_by_weekday = df['weekday'].value_counts().reindex(weekday_order)

# Plot using seaborn with categorical x-axis
plt.figure(figsize=(10,5))
sns.barplot(x=watch_by_weekday.index, y=watch_by_weekday.values)
plt.xlabel('Day of Week')
plt.ylabel('Number of Videos Watched')
plt.title('YouTube Watch Activity by Weekday')
plt.show()


In [ ]:
#Heatmap of weekday and hour
heatmap_data = df.groupby(['weekday', 'hour']).size().unstack(fill_value=0)

# Reorder weekdays
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex(weekday_order)

plt.figure(figsize=(12,6))
sns.heatmap(heatmap_data, cmap='YlGnBu')
plt.title('Watch Activity Heatmap (Weekday vs Hour)')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.show()


In [ ]:
# Top 10 channels that you watch
def get_channel(subtitles):
    if isinstance(subtitles, list) and len(subtitles) > 0:
        return subtitles[0].get('name', None)
    else:
        return None

df['channel'] = df['subtitles'].apply(get_channel)

# Now count top channels
top_channels = df['channel'].value_counts().head(10)

# Plot top channels
plt.figure(figsize=(12,6))
sns.barplot(x=top_channels.values, y=top_channels.index)
plt.xlabel('Number of Videos Watched')
plt.ylabel('Channel')
plt.title('Top 10 Most Watched YouTube Channels')
plt.show()


In [ ]:
# Total number of videos watched
total_videos = len(df)
print(f"Total videos watched: {total_videos}")

# Date range of watch history
start_date = df['time'].min()
end_date = df['time'].max()
print(f"Watch history from {start_date.date()} to {end_date.date()}")

# Average videos watched per day
videos_per_day = df.groupby(df['time'].dt.date).size()
avg_per_day = videos_per_day.mean()
print(f"Average videos watched per day: {avg_per_day:.2f}")

In [ ]:
#monthly count of watched videos
monthly_counts = df.groupby(df['time'].dt.to_period('M')).size()

monthly_counts.index = monthly_counts.index.to_timestamp()

plt.figure(figsize=(12,6))
plt.plot(monthly_counts.index, monthly_counts.values, marker='o')
plt.title('YouTube Videos Watched Over Time (Monthly)')
plt.xlabel('Month')
plt.ylabel('Number of Videos Watched')
plt.grid(True)
plt.show()


In [ ]:
# Average of watched videos in one go

# Sort by time
df_sorted = df.sort_values('time').reset_index(drop=True)

# Calculate time difference between consecutive videos (in minutes)
df_sorted['time_diff'] = df_sorted['time'].diff().dt.total_seconds().div(60)

# Define a new session if gap is more than 30 minutes
df_sorted['new_session'] = (df_sorted['time_diff'] > 30).cumsum()

# Session lengths (number of videos per session)
session_lengths = df_sorted.groupby('new_session').size()

print(f"Average videos per session: {session_lengths.mean():.2f}")
print(f"Number of sessions: {session_lengths.count()}")


In [ ]:
# watch count by hours
hourly_counts = df.groupby('hour').size()
weekday_counts = df['weekday'].value_counts().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

print("Watch counts by hour:")
print(hourly_counts)

print("\nWatch counts by weekday:")
print(weekday_counts)


In [ ]:
def busiest_watch_times(df):
    # Aggregate watch counts by hour and weekday
    hourly_counts = df.groupby('hour').size()
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday_counts = df['weekday'].value_counts().reindex(weekday_order)

    busiest_hour = hourly_counts.idxmax()
    busiest_day = weekday_counts.idxmax()

    print(f"Your busiest watch hour is: {busiest_hour}:00 - {busiest_hour+1}:00")
    print(f"Your busiest watch day is: {busiest_day}")
    return busiest_hour, busiest_day

def suggest_watch_times(busiest_hour, busiest_day):
    print(f"Based on your history, you are most active on {busiest_day}s around {busiest_hour}:00.")
    print("Consider scheduling your study or leisure watching around these times!")

def plot_moving_average(df, window=7):
    daily_counts = df.set_index('time').resample('D').size()
    moving_avg = daily_counts.rolling(window=window).mean()

    plt.figure(figsize=(12,6))
    plt.plot(daily_counts.index, daily_counts.values, label='Daily Counts')
    plt.plot(moving_avg.index, moving_avg.values, label=f'{window}-Day Moving Average', linewidth=3)
    plt.title('Daily Watch Counts with Moving Average')
    plt.xlabel('Date')
    plt.ylabel('Number of Videos Watched')
    plt.legend()
    plt.show()

def top_channels_and_videos(df, top_n=5):
    top_channels = df['channel'].value_counts().head(top_n)
    top_videos = df['title'].value_counts().head(top_n)

    print(f"Top {top_n} Channels You Watch:")
    print(top_channels)
    print("\nTop {top_n} Videos You Watch:")
    print(top_videos)
    
    return top_channels, top_videos



In [ ]:
# 1. Find busiest watch times
hour, day = busiest_watch_times(df)


In [ ]:
# 2. Suggest watch times
suggest_watch_times(hour, day)

In [ ]:
# 3. Plot moving average of daily watches
plot_moving_average(df)

In [ ]:
# 4. Print top channels and videos
top_channels, top_videos = top_channels_and_videos(df)